In [8]:
import csv
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import numpy as np
from rich.progress import Progress
from tqdm import tqdm
import optuna

In [9]:
# Define Autoencoder Model
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim, dropout=0.0, use_batch_norm=False):
        super(Autoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128) if use_batch_norm else nn.Identity(),
            nn.Dropout(dropout),
            nn.Linear(128, latent_dim),
            nn.ReLU(),
            nn.BatchNorm1d(latent_dim) if use_batch_norm else nn.Identity(),
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128) if use_batch_norm else nn.Identity(),
            nn.Dropout(dropout),
            nn.Linear(128, input_dim),
            nn.Sigmoid(),  # Assuming input values are normalized to [0, 1]
        )

    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return latent, reconstructed

In [10]:
# Custom dataset class to handle CSV input and padding
class TreeDataset(Dataset):
    def __init__(self, csv_file, padding_length=161):
        # Load data from CSV
        self.padding_length = padding_length
        self.data = self._load_csv(csv_file)

    def _min_max_normalize(self, array, min_val=0, max_val=100):
        """
        Normalize array to the range [0, 1] based on given min_val and max_val.
        """
        array = np.array(array, dtype=np.float32)
        return (array - min_val) / (max_val - min_val)

    def _load_csv(self, csv_file):
        """Read CSV file using Python's built-in csv module."""
        data = []
        with open(csv_file, "r") as csvfile:
            reader = csv.reader(csvfile)
            with Progress() as progress:
                task = progress.add_task(
                    "[cyan]Processing CSV...", total=sum(1 for _ in csvfile)
                )  # Total rows in file
                csvfile.seek(0)  # Reset file pointer

                for row in reader:
                    row = list(map(int, row))
                    normalized_row = self._min_max_normalize(row)
                    data.append(normalized_row)
                    # Update progress bar
                    progress.update(task, advance=1)
        return data

    def __len__(self):
        """Return the length of the dataset (number of rows)."""
        return len(self.data)

    def __getitem__(self, idx):
        """Retrieve a single data point from the dataset."""
        return self.data[idx]

In [11]:
def set_seed_for_everything(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [12]:
def train_model(model, criterion, optimizer, epochs, train_loader, params):
    print(f"Training the Autoencoder with params: {params}, Total epochs: {epochs}")
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        epoch_rmse = 0
        total_samples = 0
        for batch in tqdm(
            train_loader, desc=f"Epoch [{epoch+1}/{epochs}]", unit="batch"
        ):
            optimizer.zero_grad()
            _, reconstructed = model(batch)
            loss = criterion(reconstructed, batch)  # Reconstruction loss
            loss.backward()
            optimizer.step()

            # Accumulate loss
            batch_size = batch.size(0)
            total_samples += batch_size
            epoch_loss += loss.item() * batch_size  # Weighted by batch size

            # Calculate and accumulate RMSE
            rmse = torch.sqrt(loss)  # RMSE = sqrt(MSE)
            epoch_rmse += rmse.item() * batch_size  # Weighted by batch size

        # Compute average loss and RMSE over all samples
        epoch_loss /= total_samples
        epoch_rmse /= total_samples

        print(
            f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.6f}, RMSE: {epoch_rmse:.6f}"
        )

In [14]:
def validate_model(model, criterion, val_loader):
    ###
    # Validate the model
    ###
    model.eval()
    val_loss = 0
    val_rmse = 0
    total_samples = 0
    # latent_representations = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating...", unit="batch"):
            latent, reconstructed = model(batch)
            loss = criterion(reconstructed, batch)
            batch_size = batch.size(0)
            total_samples += batch_size
            val_loss += loss.item() * batch_size

            # latent_representations.append(latent)

            rmse = torch.sqrt(loss)
            val_rmse += rmse.item() * batch_size

    val_loss /= total_samples
    val_rmse /= total_samples

    # latent_representations = torch.cat(latent_representations).numpy()
    # print(f"Latent representations shape: {latent_representations.shape}")
    print(f"Validation Loss: {val_loss:.6f}, Validation RMSE: {val_rmse:.6f}")
    return val_rmse

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

set_seed_for_everything(seed=1234)

# Load dataset
csv_file = "train_data.csv"  # Replace with your CSV file path
dataset = TreeDataset(csv_file)
print(f"Dataset size: {len(dataset)}")
train_size = int(0.8 * len(dataset))
print(f"Train size: {train_size}")
val_size = len(dataset) - train_size
print(f"Validation size: {val_size}")
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

input_dim = 161  # Number of features in the dataset
epochs = 2
criterion = torch.nn.MSELoss()

Output()

Using device: cuda


Dataset size: 851229
Train size: 680983
Validation size: 170246


In [16]:
def objective(trial):
    # if you want to try suggest_float, put log=True for lr and weight_decay
    lr = trial.suggest_categorical("lr", [0.0001, 0.001, 0.01])
    weight_decay = trial.suggest_categorical("weight_decay", [0, 1e-5, 1e-4, 1e-3])
    dropout = trial.suggest_categorical("dropout", [0.0, 0.1, 0.2])
    latent_dim = trial.suggest_categorical("latent_dim", [32, 64, 128])
    use_batch_norm = True

    model = Autoencoder(
        input_dim=input_dim,
        latent_dim=latent_dim,
        dropout=dropout,
        use_batch_norm=use_batch_norm,
    )
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    train_model(model, criterion, optimizer, epochs, train_loader, trial.params)
    val_rmse = validate_model(model, criterion, val_loader)
    return val_rmse


# Run the optimization
study = optuna.create_study()
study.optimize(objective, n_trials=40)

trial = study.best_trial
best_params = trial.params  # The best hyperparameters found
best_val_rmse = trial.value  # The best value found (i.e., best validation loss)
# Print the best hyperparameters found by optuna
print(f"Best params: {best_params}, Best Validation RMSE: {best_val_rmse:.6f}")

[I 2024-11-27 00:56:18,541] A new study created in memory with name: no-name-4c6bcaf3-d9e1-4434-aaf6-6f2605d1805d


Training the Autoencoder with params: {'lr': 0.01, 'weight_decay': 0.001, 'dropout': 0.0, 'latent_dim': 32}, Total epochs: 2


Epoch [1/2]: 100%|██████████| 21281/21281 [03:10<00:00, 111.84batch/s]


Epoch [1/2], Loss: 0.006822, RMSE: 0.082032


Epoch [2/2]: 100%|██████████| 21281/21281 [03:50<00:00, 92.43batch/s]


...

Epoch [1/2], Loss: 0.001303, RMSE: 0.012491


Epoch [2/2]: 100%|██████████| 21281/21281 [00:18<00:00, 1148.07batch/s]


Epoch [2/2], Loss: 0.000030, RMSE: 0.005464


Validating...: 100%|██████████| 5321/5321 [00:00<00:00, 6394.24batch/s]
[I 2024-11-27 03:15:10,713] Trial 39 finished with value: 0.0034204485318880978 and parameters: {'lr': 0.001, 'weight_decay': 0, 'dropout': 0.2, 'latent_dim': 64}. Best is trial 19 with value: 0.0022389229271265526.


Validation Loss: 0.000012, Validation RMSE: 0.003420
Best params: {'lr': 0.001, 'weight_decay': 0, 'dropout': 0.0, 'latent_dim': 64}, Best Validation RMSE: 0.002239
